In [1]:
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00


In [2]:
import os
from datasets import load_dataset
from tokenizers import Tokenizer, pre_tokenizers, models, trainers

In [3]:
ds = load_dataset('thainq107/iwslt2015-en-vi')

tokenizer_en = Tokenizer(models.WordLevel(unk_token='<unk>'))
tokenizer_vi = Tokenizer(models.WordLevel(unk_token='<unk>'))

tokenizer_en.pre_tokenizers = pre_tokenizers.Whitespace()
tokenizer_vi.pre_tokenizers = pre_tokenizers.Whitespace()

trainer = trainers.WordLevelTrainer(
    vocab_size=15000,
    min_frequency=2,
    special_tokens=['<bos>', '<eos>', '<unk>', '<pad>']
)

tokenizer_en.train_from_iterator(ds['train']['en'], trainer=trainer)
tokenizer_vi.train_from_iterator(ds['train']['vi'], trainer=trainer)

tokenizer_en.save('tokenizer_en.json')
tokenizer_vi.save('tokenizer_vi.json')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/522 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/181k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/133317 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1268 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [4]:
from transformers import PreTrainedTokenizerFast

MAX_LEN = 75

tokenizer_en = PreTrainedTokenizerFast(tokenizer_file='tokenizer_en.json',
                                      unk_token='<unk>', pad_token='<pad>', bos_token='<bos>', eos_token='<eos>')
tokenizer_vi = PreTrainedTokenizerFast(tokenizer_file='tokenizer_vi.json',
                                       unk_token='<unk>', pad_token='<pad>', bos_token='<bos>', eos_token='<eos>')

def preprocess_function(examples):
  src_texts = examples['en']
  tgt_texts = ['<bos> ' + sent + ' <eos>' for sent in examples['vi']]

  src_encoding = tokenizer_en(
      src_texts, padding='max_length', truncation=True, max_length=MAX_LEN
  )
  tgt_encoding = tokenizer_vi(
      tgt_texts, padding='max_length', truncation=True, max_length=MAX_LEN
  )

  return {
      'input_ids': src_encoding['input_ids'],
      'labels': tgt_encoding['input_ids']
  }

preprocessed_ds = ds.map(preprocess_function, batched=True)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Map:   0%|          | 0/133317 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

Map:   0%|          | 0/1268 [00:00<?, ? examples/s]

In [14]:
import torch
import torch.nn as nn
from transformers import PreTrainedModel, PretrainedConfig

def generate_square_mask(sz, device):
  mask = (torch.triu(torch.ones((sz, sz), device=device)) == 1).transpose(0, 1)
  mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))

  return mask

def create_mask(src, tgt):
  src_seq_len = src.shape[1]
  tgt_seq_len = tgt.shape[1]

  device = src.device

  tgt_mask = generate_square_mask(tgt_seq_len, device).to(torch.bool)
  src_mask = torch.zeros((src_seq_len, src_seq_len), device=device).type(torch.bool)

  tgt_key_padding_mask = (tgt == 0)
  src_key_padding_mask = (src == 0)

  return src_mask, tgt_mask, src_key_padding_mask, tgt_key_padding_mask

class Seq2SeqTransformerConfig(PretrainedConfig):
  def __init__(self, vocab_size_src=10000, vocab_size_tgt=10000, max_seq_length=50, d_model=256, num_heads=8, num_layers=6, dropout=0.1, **kwargs):
    super().__init__(**kwargs)
    self.vocab_size_src = vocab_size_src
    self.vocab_size_tgt = vocab_size_tgt
    self.max_seq_length = max_seq_length
    self.d_model = d_model
    self.num_heads = num_heads
    self.num_layers = num_layers
    self.dropout = dropout

class Seq2SeqTransformerModel(PreTrainedModel):
  def __init__(self, config):
    super().__init__(config)
    self.embedding_src = nn.Embedding(config.vocab_size_src, config.d_model)
    self.embedding_tgt = nn.Embedding(config.vocab_size_tgt, config.d_model)

    self.position_embedding_src = nn.Embedding(config.max_seq_length, config.d_model)
    self.position_embedding_tgt = nn.Embedding(config.max_seq_length, config.d_model)

    self.transformer = nn.Transformer(d_model=config.d_model,
                                      nhead=config.num_heads,
                                      num_encoder_layers=config.num_layers,
                                      num_decoder_layers=config.num_layers,
                                      dropout=config.dropout,
                                      batch_first=True)
    self.generator = nn.Linear(config.d_model, config.vocab_size_tgt)

    self.loss_func = nn.CrossEntropyLoss(ignore_index=0)

  def forward(self, input_ids, labels):
    tgt_input = labels[:, :-1]
    tgt_output = labels[:, 1:]

    batch_size, seq_len_src = input_ids.shape
    _,seq_len_tgt = tgt_input.shape

    src_positions = torch.arange(seq_len_src, device=input_ids.device).unsqueeze(0)
    tgt_positions = torch.arange(seq_len_tgt, device=input_ids.device).unsqueeze(0)

    src_embedded = self.embedding_src(input_ids) + self.position_embedding_src(src_positions)
    tgt_embedded = self.embedding_tgt(tgt_input) + self.position_embedding_tgt(tgt_positions)

    src_mask, tgt_mask, src_key_padding_mask, tgt_key_padding_mask = create_mask(input_ids, tgt_input)

    outs = self.transformer(
        src_embedded, tgt_embedded,
        src_mask, tgt_mask,
        src_key_padding_mask=src_key_padding_mask, tgt_key_padding_mask=tgt_key_padding_mask
    )
    logits = self.generator(outs)
    loss = self.loss_func(logits.permute(0,2,1), tgt_output)

    return {'loss': loss, 'logits': logits}

  def encode(self, src, src_mask):
    batch_size, seq_leng_src = src.shape
    src_positions = torch.arange(seq_leng_src, device=src.device).unsqueeze(0)
    src_embedded = self.embedding_src(src) + self.position_embedding_src(src_positions)

    return self.transformer.encoder(src_embedded, src_mask)

  def decode(self, tgt, encoder_output, tgt_mask):
    batch_size, seq_leng_tgt = tgt.shape
    tgt_positions = torch.arange(seq_leng_tgt, device=tgt.device).unsqueeze(0)
    tgt_embedded = self.embedding_tgt(tgt) + self.position_embedding_tgt(tgt_positions)

    return self.transformer.decoder(tgt_embedded, encoder_output, tgt_mask)

config = Seq2SeqTransformerConfig(vocab_size_src=len(tokenizer_en), vocab_size_tgt=len(tokenizer_vi), max_seq_length=MAX_LEN)

model = Seq2SeqTransformerModel(config)


In [6]:
inputs_ids = torch.tensor([preprocessed_ds['train'][0]['input_ids']])
labels = torch.tensor([preprocessed_ds['train'][0]['labels']])

pred = model(inputs_ids, labels)
pred['logits'].shape

torch.Size([1, 74, 296])

In [11]:
import os
# os.environ['WANDB_DISABLED'] = 'true'

from transformers import Trainer, TrainingArguments

# Training
training_args = TrainingArguments(
    output_dir="./en-vi-machine-translation",
    logging_dir="logs",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=256,
    per_device_eval_batch_size=256,
    num_train_epochs=13,
    learning_rate=1e-4,
    save_total_limit=1,
    report_to="wandb",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=preprocessed_ds["train"],
    eval_dataset=preprocessed_ds["validation"],
)

trainer.train()

Epoch,Training Loss,Validation Loss
1,0.000600,nan
2,0.000000,nan
3,0.000000,nan
4,0.000000,nan
5,0.000000,nan
6,0.000000,nan
7,0.000000,nan
8,0.000000,nan
9,0.000000,nan
10,0.000000,nan


TrainOutput(global_step=6773, training_loss=5.312940251955271e-05, metrics={'train_runtime': 6303.8774, 'train_samples_per_second': 274.929, 'train_steps_per_second': 1.074, 'total_flos': 1.3601564804178e+16, 'train_loss': 5.312940251955271e-05, 'epoch': 13.0})

In [16]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.9 MB/s eta 0:00:00


In [19]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, device="cpu"):
    src = src.to(device)
    src_mask = src_mask.to(device)

    memory = model.encode(src, src_mask)
    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(device)

    for i in range(max_len - 1):
        memory = memory.to(device)
        tgt_mask = (generate_square_mask(ys.size(1), device)
                    .type(torch.bool)).to(device)
        out = model.decode(ys, memory, tgt_mask)
        prob = model.generator(out[:, -1, :])  # LM Head
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word[-1].item()  # index

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)

        if next_word == 3:  # EOS: 3
            break

    return ys


def translate(model, src_sentence, device='cpu'):
    model.eval()
    input_ids = tokenizer_en([src_sentence], return_tensors='pt')['input_ids'].to(device)
    num_tokens = input_ids.shape[1]
    src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool).to(device)

    tgt_tokens = greedy_decode(
        model, input_ids, src_mask, max_len=num_tokens + 5, start_symbol=2, device=device
    )

    return tokenizer_vi.decode(tgt_tokens.detach().cpu()[0])


translate(model, "I go to school", model.device)  # => "Tôi đến trường"


# Evaluate on test set
from tqdm import tqdm
import sacrebleu

pred_sentences, tgt_sentences = [], []
for sample in tqdm(ds['test']):
    src_sentence = sample['en']
    tgt_sentence = sample['vi']

    pred_sentence = translate(model, src_sentence)
    pred_sentences.append(pred_sentence)
    tgt_sentences.append(tgt_sentence)

bleu_score = sacrebleu.corpus_bleu(pred_sentences, [tgt_sentences], force=True)

100%|██████████| 1268/1268 [01:22<00:00, 15.40it/s]


In [20]:
bleu_score

BLEU = 0.12 6.8/0.2/0.0/0.0 (BP = 1.000 ratio = 1.280 hyp_len = 43192 ref_len = 33738)

In [23]:
for i in range(5):
  print(ds['test']['en'][i])

  print(pred_sentences[i])
  print(tgt_sentences[i])
  print('---------------------')

When I was little , I thought my country was the best on the planet , and I grew up singing a song called &quot; Nothing To Envy . &quot;
<unk> Con làm được không ? Con làm được không ? Cám ơn các bạn đã lắng nghe . Cám ơn các bạn đã lắng nghe . Xin cảm ơn rất nhiều
Khi tôi còn nhỏ , Tôi nghĩ rằng BắcTriều Tiên là đất nước tốt nhất trên thế giới và tôi thường hát bài &quot; Chúng ta chẳng có gì phải ghen tị . &quot;
---------------------
And I was very proud .
<unk> Con làm được không ? Con làm được không ? Cám ơn các bạn đã lắng nghe . Cám ơn các bạn đã lắng nghe . Xin cảm ơn rất nhiều
Tôi đã rất tự hào về đất nước tôi .
---------------------
In school , we spent a lot of time studying the history of Kim Il-Sung , but we never learned much about the outside world , except that America , South Korea , Japan are the enemies .
<unk> Con làm được không ? Con làm được không ? Cám ơn các bạn đã lắng nghe . Cám ơn các bạn đã lắng nghe . Xin cảm ơn rất nhiều
Ở trường , chúng tôi dành rất nhiề